In [1]:
import numpy as np
import pickle
import os

from os import listdir
from sklearn.preprocessing import LabelBinarizer
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation, Flatten, Dropout, Dense
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.preprocessing import image
from keras.preprocessing.image import img_to_array
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.utils import Bunch
import matplotlib.pyplot as plt
from pathlib import Path
from skimage.io import imread
from skimage.transform import resize
from keras.applications.mobilenet import MobileNet
from keras.optimizers import SGD
from keras.layers import Dense, GlobalAveragePooling2D
import os.path
import fnmatch
import itertools
import functools
from keras.models import Model

Using TensorFlow backend.


In [2]:
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

In [3]:
train_data_dir = 'data/train'
validation_data_dir = 'data/val'

In [4]:
train_datagen = ImageDataGenerator()
validation_datagen = ImageDataGenerator()
test_datagen = ImageDataGenerator()

In [5]:
EPOCHS = 100
INIT_LR = 1e-3
BS = 8
image_size = 0
width=224
height=224
depth=3

In [6]:
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(height, width),
    batch_size=BS,
    class_mode='categorical')

validation_generator = validation_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(height, width),
    batch_size=BS,
    class_mode='categorical')
test_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(height, width),
    batch_size=BS,
    class_mode='categorical',
    shuffle = False)

Found 2800 images belonging to 7 classes.
Found 700 images belonging to 7 classes.
Found 700 images belonging to 7 classes.


In [7]:
base_model = MobileNet(weights='imagenet', include_top=False )
# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
# and a logistic layer -- let's say we have 200 classes
predictions = Dense(7, activation='softmax')(x)
# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

In [8]:
for layer in model.layers:
    layer.trainable = True

In [9]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, None, None, 3)]   0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, None, None, 3)     0         
_________________________________________________________________
conv1 (Conv2D)               (None, None, None, 32)    864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, None, None, 32)    128       
_________________________________________________________________
conv1_relu (ReLU)            (None, None, None, 32)    0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, None, None, 32)    288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, None, None, 32)    128   

In [10]:
from tensorflow.keras.callbacks import ModelCheckpoint

checkpoint = ModelCheckpoint('best_model.h5', verbose=1, monitor='val_accuracy', save_best_only=True, mode='auto') 

In [11]:
history = model.fit_generator(
    train_generator,
    validation_data=validation_generator,
    steps_per_epoch=train_generator.n // BS,
    validation_steps=validation_generator.n // BS,
    epochs=EPOCHS, verbose=1,
    callbacks=[checkpoint],
)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/100
349/350 [============================>.] - ETA: 0s - loss: 1.1564 - accuracy: 0.6802
Epoch 00001: val_accuracy improved from -inf to 0.66092, saving model to best_model.h5
350/350 [==============================] - 51s 145ms/step - loss: 1.1547 - accuracy: 0.6804 - val_loss: 2.0767 - val_accuracy: 0.6609
Epoch 2/100
350/350 [==============================] - ETA: 0s - loss: 0.5876 - accuracy: 0.8471
Epoch 00002: val_accuracy did not improve from 0.66092
350/350 [==============================] - 51s 146ms/step - loss: 0.5876 - accuracy: 0.8471 - val_loss: 4.6445 - val_accuracy: 0.5991
Epoch 3/100
350/350 [==============================] - ETA: 0s - loss: 0.4876 - accuracy: 0.8843
Epoch 00003: val_accuracy improved from 0.66092 to 0.88362, saving model to best_model.h5
350/350 [==============================] - 51s 146ms/step - loss: 0.4876 - accuracy: 0.8843 - val_loss: 0.6767 - val_accuracy: 0.8836

Epoch 28/100
350/350 [==============================] - ETA: 0s - loss: 0.2252 - accuracy: 0.9761
Epoch 00028: val_accuracy did not improve from 0.97701
350/350 [==============================] - 50s 143ms/step - loss: 0.2252 - accuracy: 0.9761 - val_loss: 0.2359 - val_accuracy: 0.9713
Epoch 29/100
350/350 [==============================] - ETA: 0s - loss: 0.1931 - accuracy: 0.9807
Epoch 00029: val_accuracy did not improve from 0.97701
350/350 [==============================] - 50s 143ms/step - loss: 0.1931 - accuracy: 0.9807 - val_loss: 0.2445 - val_accuracy: 0.9511
Epoch 30/100
350/350 [==============================] - ETA: 0s - loss: 0.2050 - accuracy: 0.9754
Epoch 00030: val_accuracy did not improve from 0.97701
350/350 [==============================] - 51s 144ms/step - loss: 0.2050 - accuracy: 0.9754 - val_loss: 0.4579 - val_accuracy: 0.9641
Epoch 31/100
350/350 [==============================] - ETA: 0s - loss: 0.2321 - accuracy: 0.9775
Epoch 00031: val_accuracy did not improve

Epoch 57/100
350/350 [==============================] - ETA: 0s - loss: 0.1651 - accuracy: 0.9907
Epoch 00057: val_accuracy improved from 0.97701 to 0.97989, saving model to best_model.h5
350/350 [==============================] - 50s 142ms/step - loss: 0.1651 - accuracy: 0.9907 - val_loss: 0.2200 - val_accuracy: 0.9799
Epoch 58/100
350/350 [==============================] - ETA: 0s - loss: 0.0615 - accuracy: 0.9936
Epoch 00058: val_accuracy did not improve from 0.97989
350/350 [==============================] - 50s 144ms/step - loss: 0.0615 - accuracy: 0.9936 - val_loss: 0.2897 - val_accuracy: 0.9727
Epoch 59/100
350/350 [==============================] - ETA: 0s - loss: 0.1050 - accuracy: 0.9918
Epoch 00059: val_accuracy did not improve from 0.97989
350/350 [==============================] - 51s 146ms/step - loss: 0.1050 - accuracy: 0.9918 - val_loss: 0.1989 - val_accuracy: 0.9698
Epoch 60/100
350/350 [==============================] - ETA: 0s - loss: 0.0840 - accuracy: 0.9921
Epoch 

350/350 [==============================] - ETA: 0s - loss: 0.0760 - accuracy: 0.9936
Epoch 00085: val_accuracy did not improve from 0.98420
350/350 [==============================] - 52s 148ms/step - loss: 0.0760 - accuracy: 0.9936 - val_loss: 0.1173 - val_accuracy: 0.9842
Epoch 86/100
350/350 [==============================] - ETA: 0s - loss: 0.0502 - accuracy: 0.9932
Epoch 00086: val_accuracy did not improve from 0.98420
350/350 [==============================] - 51s 146ms/step - loss: 0.0502 - accuracy: 0.9932 - val_loss: 0.1707 - val_accuracy: 0.9799
Epoch 87/100
350/350 [==============================] - ETA: 0s - loss: 0.0766 - accuracy: 0.9939
Epoch 00087: val_accuracy did not improve from 0.98420
350/350 [==============================] - 51s 146ms/step - loss: 0.0766 - accuracy: 0.9939 - val_loss: 0.2659 - val_accuracy: 0.9784
Epoch 88/100
350/350 [==============================] - ETA: 0s - loss: 0.0583 - accuracy: 0.9946
Epoch 00088: val_accuracy did not improve from 0.98420

In [12]:
print("[INFO] Calculating model accuracy")
model.load_weights('best_model.h5')
scores = model.evaluate_generator(generator=test_generator, steps=test_generator.n // BS +1)
print(f"Test Accuracy: {scores[1]*100}")

[INFO] Calculating model accuracy
Instructions for updating:
Please use Model.evaluate, which supports generators.
Test Accuracy: 98.57142567634583


In [13]:
from sklearn.metrics import classification_report, confusion_matrix
Y_pred = model.predict_generator(test_generator, steps=test_generator.n // BS + 1)

y_pred = np.argmax(Y_pred, axis=1)
print()

print('Confusion Matrix')
target_names = list(test_generator.class_indices.keys())
print(target_names)
print(confusion_matrix(test_generator.classes, y_pred))
print('Classification Report')

print(classification_report(test_generator.classes, y_pred, target_names=target_names))

Instructions for updating:
Please use Model.predict, which supports generators.

Confusion Matrix
['BulBul', 'Chorui', 'Crow', 'Doel', 'Machranga', 'Shalik', 'cuckoo']
[[90  7  2  0  0  0  1]
 [ 0 91  8  0  0  0  1]
 [ 0  0 92  7  0  0  1]
 [ 0  0  1 90  8  0  1]
 [ 0  0  1  0 92  7  0]
 [ 0  0  1  0  1 90  8]
 [ 8  0  0  0  0  0 92]]
Classification Report
              precision    recall  f1-score   support

      BulBul       0.92      0.90      0.91       100
      Chorui       0.93      0.91      0.92       100
        Crow       0.88      0.92      0.90       100
        Doel       0.93      0.90      0.91       100
   Machranga       0.91      0.92      0.92       100
      Shalik       0.93      0.90      0.91       100
      cuckoo       0.88      0.92      0.90       100

    accuracy                           0.91       700
   macro avg       0.91      0.91      0.91       700
weighted avg       0.91      0.91      0.91       700

